In [1]:
import pandas as pd

In [19]:
import pandas as pd
import glob

path = './data/clusters' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [20]:
df['STD_CLSS_NM'].unique().__len__()

41

In [21]:
df

,Unnamed: 0,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,관광 민예품 및 선물용품 소매업,2.766830e+07
1,1,202004,강원,그외 기타 분류안된 오락관련 서비스업,3.796918e+06
2,2,202004,강원,그외 기타 스포츠시설 운영업,6.518359e+06
3,3,202004,강원,기타 수상오락 서비스업,4.996608e+06
4,4,202004,강원,내항 여객 운송업,2.791240e+06
...,...,...,...,...,...
688,29,202004,제주,체인화 편의점,8.628863e+09
689,30,202004,충남,버스 운송업,1.145848e+06
690,31,202004,충남,체인화 편의점,1.792784e+10
691,32,202004,충북,버스 운송업,1.068898e+06


In [22]:
cl1 = pd.read_csv('./data/clusters/cluster1.csv')
cl5 = pd.read_csv('./data/clusters/cluster5.csv')
cl2 = pd.read_csv('./data/clusters/cluster2.csv')
cl3 = pd.read_csv('./data/clusters/cluster3.csv')
cl4 = pd.read_csv('./data/clusters/cluster4.csv')

In [23]:
cl_list = [cl1,cl2,cl3,cl4,cl5]
for i in range(len(cl_list)):
    print("cluster_num : {}".format(i+1), cl_list[i].__len__())
    print("sido",cl_list[i].CARD_SIDO_NM.unique().__len__())
    print("class", cl_list[i].STD_CLSS_NM.unique().__len__())

cluster_num : 1 153
sido 17
class 9
cluster_num : 2 323
sido 17
class 19
cluster_num : 3 13
sido 13
class 1
cluster_num : 4 170
sido 17
class 10
cluster_num : 5 34
sido 17
class 2


In [28]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [24]:
submission = pd.read_csv('./data/jeju_data_ver1/submission.csv')

In [26]:
submission.columns

Index(['id', 'REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'], dtype='object')

In [37]:
sub = submission.merge(df,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='outer').drop(['AMT_x'], axis=1).rename(columns={'AMT_y':'AMT'})

In [42]:
sub.isnull().sum()

id                0
REG_YYMM          0
CARD_SIDO_NM      0
STD_CLSS_NM       0
AMT             701
dtype: int64

In [44]:
sub.fillna(0, inplace=True)

In [40]:
sub04 = sub[sub['REG_YYMM']==202004].fillna(0)

In [41]:
sub04

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,2.208881e+08
1,1,202004,강원,골프장 운영업,5.352786e+09
2,2,202004,강원,과실 및 채소 소매업,1.583986e+09
3,3,202004,강원,관광 민예품 및 선물용품 소매업,2.766830e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,3.796918e+06
...,...,...,...,...,...
692,692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.284142e+09
693,693,202004,충북,한식 음식점업,3.165686e+10
694,694,202004,충북,호텔업,2.017433e+07
695,695,202004,충북,화장품 및 방향제 소매업,7.405451e+08


In [83]:
sub.to_csv('submission_3.csv', index=False, encoding='utf-8-sig')

In [54]:
df_raw = pd.read_csv('./data/jeju_data_ver1/201901-202003.csv')

In [55]:
df=df_raw

In [56]:
df.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [57]:
df1 = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index()

In [58]:
sido_list = list(df_raw['CARD_SIDO_NM'].unique())
clss_list = list(df_raw['STD_CLSS_NM'].unique())

In [59]:
dict1 = dict()
for i in sido_list:
    res = df1[df1['CARD_SIDO_NM'] =='{}'.format(i)]['STD_CLSS_NM'].unique()
    result = set(clss_list) - set(res)
    dict1['{}'.format(i)] = list(result)

In [60]:
dict1

{'강원': ['정기 항공 운송업'],
 '경기': ['정기 항공 운송업', '면세점'],
 '경남': ['정기 항공 운송업', '면세점'],
 '경북': ['정기 항공 운송업', '면세점'],
 '광주': ['정기 항공 운송업', '휴양콘도 운영업', '면세점', '그외 기타 분류안된 오락관련 서비스업'],
 '대구': ['정기 항공 운송업', '내항 여객 운송업', '휴양콘도 운영업', '그외 기타 분류안된 오락관련 서비스업'],
 '대전': ['택시 운송업',
  '내항 여객 운송업',
  '버스 운송업',
  '휴양콘도 운영업',
  '그외 기타 분류안된 오락관련 서비스업',
  '정기 항공 운송업',
  '면세점'],
 '부산': ['그외 기타 분류안된 오락관련 서비스업'],
 '서울': ['휴양콘도 운영업', '그외 기타 분류안된 오락관련 서비스업'],
 '세종': ['여행사업',
  '택시 운송업',
  '내항 여객 운송업',
  '버스 운송업',
  '휴양콘도 운영업',
  '호텔업',
  '정기 항공 운송업',
  '자동차 임대업',
  '면세점'],
 '울산': ['정기 항공 운송업', '내항 여객 운송업', '휴양콘도 운영업'],
 '인천': ['휴양콘도 운영업', '버스 운송업', '그외 기타 분류안된 오락관련 서비스업'],
 '전남': ['정기 항공 운송업'],
 '전북': ['면세점', '그외 기타 분류안된 오락관련 서비스업'],
 '제주': ['버스 운송업'],
 '충남': ['면세점'],
 '충북': ['정기 항공 운송업', '그외 기타 분류안된 오락관련 서비스업']}

In [80]:
for idx in range(len(sub)):
    for k,v in dict1.items():
        if sub.iloc[idx,2] == k:
            for clss in v:
                if sub.iloc[idx,3] == clss:
                    sub.iloc[idx,4] = 0

In [81]:
sub[sub['STD_CLSS_NM']=='정기 항공 운송업']

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
30,30,202004,강원,정기 항공 운송업,0.000000e+00
71,71,202004,경기,정기 항공 운송업,0.000000e+00
112,112,202004,경남,정기 항공 운송업,0.000000e+00
153,153,202004,경북,정기 항공 운송업,0.000000e+00
194,194,202004,광주,정기 항공 운송업,0.000000e+00
235,235,202004,대구,정기 항공 운송업,0.000000e+00
276,276,202004,대전,정기 항공 운송업,0.000000e+00
317,317,202004,부산,정기 항공 운송업,6.700440e+08
358,358,202004,서울,정기 항공 운송업,5.943864e+09
399,399,202004,세종,정기 항공 운송업,0.000000e+00


In [ ]:
sub